<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Profile_Photos/blob/main/IVY_Cornell_Johnson_Meta_and_Smile_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta Dataset

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

df_names =[]
df_urls = []
df_schools = []
df_departments = []

departments = ["accounting", "finance", "management-and-organizations", "marketing", "operations-technology-information-management", "economics"]

for department in departments:
  url = f"https://www.johnson.cornell.edu/programs/phd-program/{department}/"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  #print(url, response)

  sub_divs = soup.find_all("div", class_="g-col t-content site-panel--wysiwyg__col")

  for sub_div in sub_divs:
      p_tags = sub_div.find_all("a")
      for p_tag in p_tags:
          name = p_tag.get_text()
          href = p_tag["href"]
          school = p_tag.find_next('br').next_sibling.strip()

          print(name, school)
          df_names.append(name)
          df_urls.append(href)
          df_schools.append(school)
          df_departments.append(department)

Sanjeev Bhojraj Johnson School
Robert J. Bloomfield Johnson School
Nicholas Matthew Guest Johnson School
Ryan Guggenmos Johnson School
Robert Libby Johnson School
Yao Lu Johnson School
Asis Martinez-Jerez Nolan School
Mark W. Nelson Johnson School
Michael Paz Nolan School
Kristina Marie Rennekamp Johnson School
Mani Sethuraman Johnson School
Emma Wang Dyson School
P. Eric Yeung Johnson School
Xinyu Zhang Johnson School
Jawad Addoum Dyson School
Warren B. Bailey Johnson
Matthew Baron Johnson
Vicki Bogan Dyson
Murillo Campello Johnson
Linda Canina Nolan
Steven Carvell Nolan
Will Cong Johnson School
Adrian Aycan Corum Johnson School
Minmo Gahng Dyson School
Ming Huang Johnson School
Robert A. Jarrow Johnson School
Andrew Karolyi Johnson School
Crocker Liu Nolan School
Yifei Mao Nolan School
Pamela C. Moulton Nolan School
Justin Murfin Dyson School
David Ng Dyson School
Maureen O’Hara Johnson School
Gideon Saar Johnson School
Scott Yonker Dyson School
Gautam Ahuja Johnson School
Ya-Ru Chen

In [4]:
df_1 = pd.DataFrame({"Name":df_names,
                     "School":df_schools,
                     "Department":df_departments,
                     "URL":df_urls})

df_1.head(3)

,Name,School,Department,URL
0,Sanjeev Bhojraj,Johnson School,accounting,https://www.johnson.cornell.edu/faculty-resear...
1,Robert J. Bloomfield,Johnson School,accounting,https://www.johnson.cornell.edu/faculty-resear...
2,Nicholas Matthew Guest,Johnson School,accounting,https://www.johnson.cornell.edu/faculty-resear...


In [5]:
df_1["School"].value_counts()

Johnson School    55
Nolan School      27
Dyson School      21
Johnson            3
Nolan              2
Dyson              1
ILR School         1
Name: School, dtype: int64

## Filter the dataset based on School

In [7]:
df_1_J = df_1[df_1["School"].str.contains("Johnson")]

df_1_J.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 108
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        58 non-null     object
 1   School      58 non-null     object
 2   Department  58 non-null     object
 3   URL         58 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [8]:
df_titles = []

for index, row in df_1_J.iterrows():
    name = row["Name"]
    url = row["URL"]
    response = requests.get(url)
    #print(name, url, response)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find("ul", class_="faculty-profile__list")
    title = title.get_text().strip() if title else None

    df_titles.append(title)
    #print(name, ":", title, "\n")



df_1_J["Title"] = df_titles

<ipython-input-8-2853a42ce968>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_J["Title"] = df_titles


In [9]:
df_1_J = df_1_J[df_1_J["Name"] != "Lawrence W. Robinson"] #No page exist

In [10]:
df_1_J["School"].value_counts()

Johnson School    54
Johnson            3
Name: School, dtype: int64

## Rank

In [11]:
import numpy as np

conditions = [
    df_1_J["Title"].str.contains("Emeritus"),
    df_1_J["Title"].str.contains("Emerita"),
    df_1_J["Title"].str.contains("Assistant"),
    df_1_J["Title"].str.contains("Associate")
]

choices = ["Emeritus", "Emeritus", "Assistant", "Associate"]

df_1_J["Rank"] = np.select(conditions, choices, default="Full")

df_1_J["Rank"].value_counts()

<ipython-input-11-732a2998eda2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_J["Rank"] = np.select(conditions, choices, default="Full")


Full         29
Assistant    19
Associate     9
Name: Rank, dtype: int64

In [12]:
df_1_J.groupby("Department")["Rank"].value_counts()

Department                                    Rank     
accounting                                    Full         6
                                              Assistant    5
economics                                     Assistant    4
                                              Full         2
                                              Associate    1
finance                                       Full         7
                                              Assistant    2
                                              Associate    1
management-and-organizations                  Full         6
                                              Associate    3
                                              Assistant    1
marketing                                     Full         5
                                              Assistant    3
                                              Associate    3
operations-technology-information-management  Assistant    4
                             

## Sex Information based on Biography

In [13]:
df_bios = []

for index, row in df_1_J.iterrows():
    name = row["Name"]
    url = row["URL"]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    p_tag = soup.find("p", class_="faculty-profile__bio")
    bio = p_tag.get_text().strip() if p_tag else None

    df_bios.append(bio)

    #print(name, bio, "\n")

df_1_J["Biography"] = df_bios

In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df_1_J['Biography']:
    if text == None:
      text = ""
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df_1_J['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df_1_J['Sex'] = df_1_J['Pronouns'].apply(determine_sex)

### No Biography information case handlings

In [15]:
df_1_J[df_1_J["Sex"] == "Unknown"]

,Name,School,Department,URL,Title,Rank,Biography,Pronouns,Sex
46,Elizabeth McClean,Johnson School,management-and-organizations,https://www.johnson.cornell.edu/faculty-resear...,Associate Professor,Associate,None,[],Unknown
73,Omid Rafieian,Johnson School,marketing,https://www.johnson.cornell.edu/faculty-resear...,Assistant Professor\nDemir Sabanci Faculty Fel...,Assistant,None,[],Unknown
89,Yichun Hu,Johnson School,operations-technology-information-management,https://business.cornell.edu/faculty-research/...,Assistant Professor,Assistant,None,[],Unknown
90,Nur Kaynar,Johnson School,operations-technology-information-management,https://www.johnson.cornell.edu/faculty-resear...,Assistant Professor,Assistant,None,[],Unknown
97,Adam Dearing,Johnson School,economics,https://www.johnson.cornell.edu/faculty-resear...,Assistant Professor,Assistant,None,[],Unknown


In [16]:
# https://kjzz.org/content/1685853/pitching-idea-work-university-arizona-study-says-go-against-gender-stereotypes
# https://tech.cornell.edu/people/omid-rafieian/
# https://business.cornell.edu/hub/2023/07/17/meet-our-new-faculty-yichun-hu/
# https://www.cornell.edu/video/nur-kaynar-financial-incentives-worker-productivity-online-labor-platforms


df_1_J.loc[df_1_J["Name"] =="Elizabeth McClean", "Sex"] = "Female"
df_1_J.loc[df_1_J["Name"] =="Omid Rafieian", "Sex"] = "Male"
df_1_J.loc[df_1_J["Name"] =="Yichun Hu", "Sex"] = "Female"
df_1_J.loc[df_1_J["Name"] =="Nur Kaynar", "Sex"] = "Female"


In [17]:
df_1_J[df_1_J["Sex"] == "Unknown"]

,Name,School,Department,URL,Title,Rank,Biography,Pronouns,Sex
97,Adam Dearing,Johnson School,economics,https://www.johnson.cornell.edu/faculty-resear...,Assistant Professor,Assistant,None,[],Unknown


# Photo scrap

In [18]:
from google.colab import files
import requests
from bs4 import BeautifulSoup

for index, row in df_1_J.iterrows():
    name = row["Name"]
    new_name = name.lower().replace(".", "").replace(" ", "_")
    filename = f"{new_name}.jpg"

    url = row["URL"]
    response = requests.get(url)
    #print(name, new_name, response)
    soup = BeautifulSoup(response.content, 'html.parser')

    div_tag = soup.find("div", class_="faculty-profile t-content")
    img_tag = div_tag.find("img")
    img_src = img_tag["src"]

    #print(name, img_src)

    flag = requests.get(img_src)

    with open(filename, "wb") as f:
        noop = f.write(flag.content)
        print(f"{name} saved the {filename}")

        files.download(filename)
        #print(f"download the {filename}")


Sanjeev Bhojraj saved the sanjeev_bhojraj.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Robert J. Bloomfield saved the robert_j_bloomfield.jpg


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Michael Waldman saved the michael_waldman.jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Smile Detection

In [30]:
import cv2
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

names = []
face_detected = []
smile_detected = []

for index, row in df_1_J.iterrows():
    name = row["Name"]
    new_name = name.lower().replace(".", "").replace(" ", "_")
    filename = f"{new_name}.jpg"

    img = cv2.imread(f'/content/{filename}', -1)

    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)

    if len(faces) > 0:
        face_var = 1
    else:
        face_var = 0

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_bottom_half = img[y + int(h * 0.4):y + h, x:x + w]

        # Check if the region of interest is not None
        if roi_bottom_half is not None:
            smiles = smile_cascade.detectMultiScale(roi_bottom_half, scaleFactor=1.3, minNeighbors=40)

            if len(smiles) > 0:
                smile_var = 1
            else:
                smile_var = 0

            smile_coordinates = [(x + sx, y + int(h * 0.4) + sy, sw, sh) for (sx, sy, sw, sh) in smiles]
            # Calculate the bounding box for all smiles
            if smile_coordinates:
                min_x = min([x for (x, y, w, h) in smile_coordinates])
                min_y = min([y for (x, y, w, h) in smile_coordinates])
                max_w = max([x + w for (x, y, w, h) in smile_coordinates]) - min_x
                max_h = max([y + h for (x, y, w, h) in smile_coordinates]) - min_y

                # Draw a single larger rectangle around all smiles
                cv2.rectangle(img, (min_x, min_y), (min_x + max_w, min_y + max_h), (0, 255, 0), 1)

    face_detected.append(face_var)
    smile_detected.append(smile_var)
    names.append(name)


    #print(name)
    #cv2_imshow(img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Wrong face recognition Michelle M. Duguid

In [20]:
df_smile = pd.DataFrame({"Name":names,
                         "Face":face_detected,
                         "Smile":smile_detected
                         })

df_smile.head()

df_smile.loc[df_smile["Name"]=="Michelle M. Duguid", "Face"] = 0

In [31]:
df_smile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    57 non-null     object
 1   Face    57 non-null     int64 
 2   Smile   57 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ KB


In [21]:
df_smile["Face"].value_counts()

1    54
0     3
Name: Face, dtype: int64

In [22]:
df_smile["Smile"].value_counts()

1    38
0    19
Name: Smile, dtype: int64

In [24]:
df_result = df_1_J.merge(df_smile, on="Name")

df_result["University"] = "Cornell"

df_result = df_result[["University","Name","Department","Rank","Sex","Title","Pronouns","URL","Face","Smile"]]

In [25]:
df_result.head(3)

,University,Name,Department,Rank,Sex,Title,Pronouns,URL,Face,Smile
0,Cornell,Sanjeev Bhojraj,accounting,Full,Male,Alumni Professor in Asset Management\nFaculty ...,"[he, his, he, he, he, he, his]",https://www.johnson.cornell.edu/faculty-resear...,1,0
1,Cornell,Robert J. Bloomfield,accounting,Full,Male,Nicholas H. Noyes Professor of Management\nAre...,[he],https://www.johnson.cornell.edu/faculty-resear...,1,1
2,Cornell,Nicholas Matthew Guest,accounting,Assistant,Male,Assistant Professor,"[his, he]",https://www.johnson.cornell.edu/faculty-resear...,1,1
